4 Versões


1.   Versão sequencial utilizada para dataset de mil linhas
2.   Versão teste para processamento em lotes no dataset real
3.   Versão com threads
4.   Versão com OpenMP



In [ ]:
!apt-get update
!apt-get install -y build-essential
!apt-get install -y libomp-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,089 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.5 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,523 kB]
Get:13 http://security.ubuntu.c

In [ ]:
from google.colab import drive #conecta Google colab ao Drive
drive.mount('/content/drive')

Mounted at /content/drive


INFORMAÇÕES DO AMBIENTE


In [ ]:
!cat /proc/cpuinfo #CPU
!cat /proc/meminfo #Memória
!df -h #Disco


processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
ad

#VERSÃO SEQUENCIAL (apenas teste dataset 1000 linhas)

In [ ]:
%%writefile no_thread.cpp
#include <iostream>
#include <fstream>         //  manipulação de arquivos
#include <sstream>         //  strings em fluxo, streams são sequencias de dados em continuidade
#include <vector>          //  utilização de vetores
#include <unordered_map>   //  utilização de mapas desordenados (hash maps)
#include <set>             //  conjuntos
#include <algorithm>       //  algoritmos padrões

// função para ler o CSV
std::vector<std::vector<std::string>> ler_csv(const std::string &caminho, std::vector<std::string> &cabecalho) {
    std::ifstream arquivo(caminho);  // abre o arquivo CSV para leitura
    std::vector<std::vector<std::string>> dados;  // vetor para armazenar os dados do CSV
    std::string linha, celula;

    // ler o cabeçalho
    if (std::getline(arquivo, linha)) {  // lê a primeira linha do arquivo (cabeçalho)
        std::stringstream linhaStream(linha);  // cria um stream a partir da linha lida
        while (std::getline(linhaStream, celula, ',')) {  // lê cada célula separada por vírgula
            cabecalho.push_back(celula);  // adiciona a célula ao vetor de cabeçalho
        }
    }

    // Ler os dados
    while (std::getline(arquivo, linha)) {  // lê cada linha restante do arquivo
        std::vector<std::string> linha_dados;  // vetor para armazenar os dados da linha atual
        std::stringstream linhaStream(linha);  // cria um stream a partir da linha lida
        while (std::getline(linhaStream, celula, ',')) {  // lê cada célula separada por vírgula
            linha_dados.push_back(celula);  // adiciona a célula ao vetor de dados da linha
        }
        dados.push_back(linha_dados);  // adiciona a linha de dados ao vetor de dados
    }

    return dados;
}

// função para verificar se uma string é numérica
bool eh_numerico(const std::string &str) {
    return !str.empty() && std::all_of(str.begin(), str.end(), ::isdigit);  // Verifica se todos os caracteres da string são dígitos
}

// Função para fatorizar colunas nominais
std::vector<std::vector<std::string>> fatorizar_colunas(const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, std::vector<std::string> &novo_cabecalho) {
    std::vector<std::vector<std::string>> novos_dados(dados.size(), std::vector<std::string>(cabecalho.size()));  // cria um vetor para armazenar os novos dados

    for (size_t col = 0; col < cabecalho.size(); ++col) {  // Itera sobre cada coluna do cabeçalho
        // Verificar se a coluna é nominal
        bool eh_nominal = std::any_of(dados.begin(), dados.end(), [col](const std::vector<std::string> &linha) {
            return !linha[col].empty() && !eh_numerico(linha[col]);  // Verifica se a coluna contém valores não numéricos
        });

        if (eh_nominal) {  // Se a coluna for nominal
            std::unordered_map<std::string, int> mapeamento;  // cria um mapa não ordenado para armazenar o mapeamento de valores
            int id = 0;

            for (size_t lin = 0; lin < dados.size(); ++lin) {  // itera sobre cada linha dos dados
                if (!dados[lin][col].empty()) {  // verifica se a célula não está vazia
                    if (mapeamento.find(dados[lin][col]) == mapeamento.end()) {  // se o valor ainda não estiver no mapa
                        mapeamento[dados[lin][col]] = id++;  // adiciona o valor ao mapa com um novo ID
                    }
                    novos_dados[lin][col] = std::to_string(mapeamento[dados[lin][col]]);  // Atribui o ID ao novo vetor de dados
                }
            }

            // Salvar a coluna categorizada em um arquivo CSV individual
            std::string nome_arquivo = "dataset_" + cabecalho[col] + ".csv";  // nomeaia arquivos
            std::ofstream arquivo(nome_arquivo);  // abre o arquivo para escrita
            arquivo << cabecalho[col] << "\n";  // escreve o cabeçalho no arquivo
            for (size_t lin = 0; lin < dados.size(); ++lin) {  // Itera sobre cada linha dos dados
                arquivo << novos_dados[lin][col] << "\n";  // Escreve o dado no arquivo
            }
            arquivo.close();
            std::cout << "Coluna '" << cabecalho[col] << "' fatorizada e salva em '" << nome_arquivo << "'\n";
        } else {  // Se a coluna não for nominal copia a coluna sem modificação

            for (size_t lin = 0; lin < dados.size(); ++lin) {  // Itera sobre cada linha dos dados
                novos_dados[lin][col] = dados[lin][col];  // copia o dado original para o novo vetor de dados
            }
        }
        novo_cabecalho.push_back(cabecalho[col]);  // Adiciona o cabeçalho da coluna ao novo cabeçalho
    }

    return novos_dados;  // retorna os novos dados fatorizados
}

// Função para salvar unificado em um arquivo CSV
void salvar_csv_unificado(const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, const std::string &nome_arquivo) {
    std::ofstream arquivo(nome_arquivo);  // abre o arquivo pra escrita

    // Escrever o cabeçalho
    for (size_t i = 0; i < cabecalho.size(); ++i) {  // itera sobre cada coluna do cabeçalho
        arquivo << cabecalho[i];  // escreve o cabeçalho no arquivo
        if (i < cabecalho.size() - 1) {  // adiciona uma vírgula se não for a última coluna
            arquivo << ",";
        }
    }
    arquivo << "\n";  // adiciona uma nova linha após o cabeçalho

    // escreve os dados
    for (const auto &linha : dados) {  // itera sobre cada linha dos dados
        for (size_t i = 0; i < linha.size(); ++i) {  // itera sobre cada célula da linha
            arquivo << linha[i];  // escreve o dado no arquivo
            if (i < linha.size() - 1) {  // adiciona uma vírgula se não for a última célula
                arquivo << ",";
            }
        }
        arquivo << "\n";
    }

    arquivo.close();
}


int main() {
    std::string caminho = "/content/drive/MyDrive/Atividades_IA/dataset_00_sem_virg.csv";
    std::vector<std::string> cabecalho;  // vetor para armazenar o cabeçalho
    auto dados = ler_csv(caminho, cabecalho);  // lê o CSV e armazena os dados

    std::vector<std::string> novo_cabecalho;  // vetor para armazenar o novo cabeçalho
    auto novos_dados = fatorizar_colunas(dados, cabecalho, novo_cabecalho);  // fatoriza as colunas nominais

    // salva unificado em um CSV
    salvar_csv_unificado(novos_dados, novo_cabecalho, "merged_dataset.csv");

    std::cout << "arquivo unificado salvo em 'merged_dataset.csv'\n";
    return 0;
}


Overwriting no_thread.cpp


In [ ]:
!g++ -o no_thread no_thread.cpp
!./no_thread

^C


# VERSÃO TESTE LEITURA EM LOTES



In [ ]:
%%writefile lotes.cpp
#include <iostream>
#include <fstream>   //  manipulação de arquivos
#include <sstream>   //  manipulação de strings em fluxo de dados
#include <vector>    // vetores
#include <unordered_map> //  mapas não ordenados
#include <set>       //  conjuntos
#include <algorithm> //  algoritmos padrão
#include <cmath>     //  funções matemáticas

// função para ler o CSV em lotes
std::vector<std::vector<std::string>> ler_csv_em_lotes(std::ifstream &arquivo, int tamanho_lote, std::vector<std::string> &cabecalho) {
    std::vector<std::vector<std::string>> dados; // vetor para armazenar os dados lidos
    std::string linha, celula; // strings para armazenar a linha e a célula atual

    // ler o cabeçalho
    if (std::getline(arquivo, linha)) { // lê a primeira linha do arquivo
        std::stringstream linhaStream(linha); // cria um stringstream com a linha lida
        while (std::getline(linhaStream, celula, ',')) { // lê cada célula da linha
            cabecalho.push_back(celula); // adiciona a célula ao cabeçalho
        }
    }

    // ler os dados em lotes
    int linhas_lidas = 0; // contador de linhas lidas
    while (linhas_lidas < tamanho_lote && std::getline(arquivo, linha)) { // lê as linhas até atingir o tamanho do lote
        std::vector<std::string> linha_dados; // vetor para armazenar os dados da linha atual
        std::stringstream linhaStream(linha); // cria um stringstream com a linha lida
        while (std::getline(linhaStream, celula, ',')) { // lê cada célula da linha
            linha_dados.push_back(celula); // adiciona a célula ao vetor de dados da linha
        }
        dados.push_back(linha_dados); // adiciona os dados da linha ao vetor de dados
        linhas_lidas++; // incrementa o contador de linhas lidas
    }

    return dados; // retorna os dados lidos
}

// função para verificar se uma string é numérica
bool eh_numerico(const std::string &str) {
    return !str.empty() && std::all_of(str.begin(), str.end(), ::isdigit); // verifica se a string não está vazia e todos os caracteres são dígitos
}

// função para fatorizar colunas nominais em lotes
std::vector<std::vector<std::string>> fatorizar_colunas_em_lotes(const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, std::vector<std::string> &novo_cabecalho) {
    std::vector<std::vector<std::string>> novos_dados(dados.size(), std::vector<std::string>(cabecalho.size())); // vetor para armazenar os novos dados

    for (size_t col = 0; col < cabecalho.size(); ++col) { // itera sobre as colunas
        // verificar se a coluna é nominal
        bool eh_nominal = std::any_of(dados.begin(), dados.end(), [col](const std::vector<std::string> &linha) {
            return !linha[col].empty() && !eh_numerico(linha[col]); // verifica se a célula não está vazia e não é numérica
        });

        if (eh_nominal) { // se a coluna for nominal
            std::unordered_map<std::string, int> mapeamento; // mapa para armazenar valores nominais para inteiros
            int id = 0; // identificador inicial

            for (size_t lin = 0; lin < dados.size(); ++lin) { // itera sobre as linhas
                if (!dados[lin][col].empty()) { // se a célula não estiver vazia
                    if (mapeamento.find(dados[lin][col]) == mapeamento.end()) { // se o valor nominal não estiver no mapa
                        mapeamento[dados[lin][col]] = id++; // adiciona o valor nominal ao mapa com um novo id
                    }
                    novos_dados[lin][col] = std::to_string(mapeamento[dados[lin][col]]); // substitui o valor nominal pelo id
                }
            }

            // salvar a coluna categorizada em um arquivo CSV individual
            std::string nome_arquivo = "dataset_" + cabecalho[col] + ".csv"; // cria o nome do arquivo
            std::ofstream arquivo(nome_arquivo); // abre o arquivo para escrita
            arquivo << cabecalho[col] << "\n"; // escreve o cabeçalho da coluna no arquivo
            for (size_t lin = 0; lin < dados.size(); ++lin) { // itera sobre as linhas
                arquivo << novos_dados[lin][col] << "\n"; // escreve os dados da coluna no arquivo
            }
            arquivo.close(); // fecha o arquivo
            std::cout << "coluna '" << cabecalho[col] << "' fatorizada e salva em '" << nome_arquivo << "'\n";
        } else { // se a coluna não for nominal
            // copiar a coluna sem modificação
            for (size_t lin = 0; lin < dados.size(); ++lin) { // itera sobre as linhas
                novos_dados[lin][col] = dados[lin][col]; // copia o valor da célula
            }
        }
        novo_cabecalho.push_back(cabecalho[col]); // adiciona o nome da coluna ao novo cabeçalho
    }

    return novos_dados;
}

// função para salvar unificado em um arquivo CSV
void salvar_csv_unificado(const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, const std::string &nome_arquivo) {
    std::ofstream arquivo(nome_arquivo); // abre o arquivo para escrita

    // escrever o cabeçalho
    for (size_t i = 0; i < cabecalho.size(); ++i) { // itera sobre o cabeçalho
        arquivo << cabecalho[i]; // escreve o nome da coluna no arquivo
        if (i < cabecalho.size() - 1) { // se não for o último elemento
            arquivo << ",";
        }
    }
    arquivo << "\n"; // adiciona uma nova linha

    // escrever os dados
    for (const auto &linha : dados) { // itera sobre as linhas
        for (size_t i = 0; i < linha.size(); ++i) { // itera sobre as células da linha
            arquivo << linha[i]; // escreve o valor da célula no arquivo
            if (i < linha.size() - 1) { // se não for o último elemento
                arquivo << ","; // adiciona uma vírgula
            }
        }
        arquivo << "\n";
    }

    arquivo.close();
}

int main() {
    std::string caminho = "/content/drive/MyDrive/Atividades_IA/dataset_00_sem_virg.csv";
    std::ifstream arquivo(caminho); // abre o arquivo para leitura
    std::vector<std::string> cabecalho; // vetor para armazenar o cabeçalho

    // tamanho do lote para leitura em lotes
    int tamanho_lote = 10000; // define o tamanho do lote

    // loop para processar o arquivo em lotes
    while (arquivo) { // enquanto o arquivo estiver aberto
        // ler e processar o próximo lote de dados
        auto dados_lote = ler_csv_em_lotes(arquivo, tamanho_lote, cabecalho); // lê um lote de dados do arquivo
        if (dados_lote.empty()) { // se não houver mais dados
            break; // sai do loop
        }

        std::vector<std::string> novo_cabecalho; // vetor para armazenar o novo cabeçalho
        auto novos_dados = fatorizar_colunas_em_lotes(dados_lote, cabecalho, novo_cabecalho); // fatoriza as colunas nominais do lote

        // salvar o arquivo unificado em um arquivo CSV
        salvar_csv_unificado(novos_dados, novo_cabecalho, "merged_dataset.csv");
    }

    std::cout << "arquivo unificado salvo em 'merged_dataset.csv'\n";
    return 0;
}


Writing lotes.cpp


In [ ]:
!g++ -o lotes lotes.cpp
!./lotes

coluna 'cdtup' fatorizada e salva em 'dataset_cdtup.csv'
coluna 'berco' fatorizada e salva em 'dataset_berco.csv'
coluna 'portoatracacao' fatorizada e salva em 'dataset_portoatracacao.csv'
coluna 'mes' fatorizada e salva em 'dataset_mes.csv'
coluna 'tipooperacao' fatorizada e salva em 'dataset_tipooperacao.csv'
coluna 'tiponavegacaoatracacao' fatorizada e salva em 'dataset_tiponavegacaoatracacao.csv'
coluna 'terminal' fatorizada e salva em 'dataset_terminal.csv'
coluna 'origem' fatorizada e salva em 'dataset_origem.csv'
coluna 'destino' fatorizada e salva em 'dataset_destino.csv'
coluna 'cdmercadoria' fatorizada e salva em 'dataset_cdmercadoria.csv'
coluna 'naturezacarga' fatorizada e salva em 'dataset_naturezacarga.csv'
coluna 'sentido' fatorizada e salva em 'dataset_sentido.csv'
coluna 'cdmercadoria_cntr' fatorizada e salva em 'dataset_cdmercadoria_cntr.csv'
.csv'
coluna 'cdtup' fatorizada e salva em 'dataset_cdtup.csv'
coluna 'berco' fatorizada e salva em 'dataset_berco.csv'
coluna 

EXCLUIR ARQUIVOS DO AMBIENTE DE EXECUÇÃO

In [ ]:
#excluir arquivos do ambiente de execução
!rm -rf dataset*


#VERSÃO THREADS CONVENCIONAIS

In [ ]:
%%writefile thread.cpp
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <unordered_map>
#include <set>
#include <algorithm>
#include <thread>
#include <mutex>

// Função para ler o CSV em lotes
std::vector<std::vector<std::string>> ler_csv_em_lotes(std::ifstream &arquivo, int tamanho_lote) {
    std::vector<std::vector<std::string>> dados;
    std::string linha, celula;
    int linhas_lidas = 0;

    while (linhas_lidas < tamanho_lote && std::getline(arquivo, linha)) {
        std::vector<std::string> linha_dados;
        std::stringstream linhaStream(linha);
        while (std::getline(linhaStream, celula, ',')) {
            linha_dados.push_back(celula);
        }
        dados.push_back(linha_dados);
        linhas_lidas++;
    }

    return dados;
}

// função para verificar se uma string é numérica
bool eh_numerico(const std::string &str) {
    char* endptr = 0;
    strtod(str.c_str(), &endptr);
    return *endptr == '\0' && !str.empty();
}

// função para fatorizar colunas nominais em lotes
void fatorizar_colunas_em_lotes(int &id, std::unordered_map<std::string, int> &mapeamento, const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, std::vector<std::vector<std::string>> &novos_dados, std::mutex &m) {
    for (size_t col = 0; col < cabecalho.size(); ++col) {
        // Verificar se a coluna é nominal
        bool eh_nominal = std::any_of(dados.begin(), dados.end(), [col](const std::vector<std::string> &linha) {
            return !linha[col].empty() && !eh_numerico(linha[col]);
        });

        if (eh_nominal) {


            for (size_t lin = 0; lin < dados.size(); ++lin) {
                if (!dados[lin][col].empty()) {
                    std::lock_guard<std::mutex> guard(m);
                    if (mapeamento.find(dados[lin][col]) == mapeamento.end()) {
                        mapeamento[dados[lin][col]] = id++;
                    }
                    novos_dados[lin][col] = std::to_string(mapeamento[dados[lin][col]]);
                }
            }

                     // Sincronizar o acesso ao arquivo CSV individual
            std::string nome_arquivo = "dataset_" + cabecalho[col] + ".csv";
            std::lock_guard<std::mutex> guard(m);
            std::ofstream arquivo(nome_arquivo, std::ios::app); //abre arquivo para adicionar valores novos sem modificar originais
            for (size_t lin = 0; lin < dados.size(); ++lin) {
                arquivo << novos_dados[lin][col] << "\n";
            }
            arquivo.close();
            std::cout << "Coluna '" << cabecalho[col] << "' fatorizada e salva em '" << nome_arquivo << "'\n";
        } else {
            for (size_t lin = 0; lin < dados.size(); ++lin) {
                novos_dados[lin][col] = dados[lin][col];
            }
        }
    }
}

// Função para salvar unificado em um arquivo CSV
void salvar_csv_unificado(const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, const std::string &nome_arquivo, bool escrever_cabecalho) {
    std::ofstream arquivo(nome_arquivo, std::ios::app);

    // Escrever o cabeçalho se necessário
    if (escrever_cabecalho) {
        for (size_t i = 0; i < cabecalho.size(); ++i) {
            arquivo << cabecalho[i];
            if (i < cabecalho.size() - 1) {
                arquivo << ",";
            }
        }
        arquivo << "\n";
    }

    // Escrever os dados
    for (const auto &linha : dados) {
        for (size_t i = 0; i < linha.size(); ++i) {
            arquivo << linha[i];
            if (i < linha.size() - 1) {
                arquivo << ",";
            }
        }
        arquivo << "\n";
    }

    arquivo.close();
}

int main() {
    std::string caminho = "/content/drive/MyDrive/Atividades_IA/dataset_00_sem_virg.csv";
    std::ifstream arquivo(caminho);
    std::vector<std::string> cabecalho;


    int tamanho_lote = 10000;
    bool escrever_cabecalho = true;

    // ler o cabeçalho
    std::string linha;
    if (std::getline(arquivo, linha)) {
        std::stringstream linhaStream(linha);
        std::string celula;
        while (std::getline(linhaStream, celula, ',')) {
            cabecalho.push_back(celula);
        }
    }

    std::mutex m;

    int id = 0;
    std::unordered_map<std::string, int> mapeamento;

    // loop para processar o arquivo em lotes
    while (arquivo) {
        auto dados_lote = ler_csv_em_lotes(arquivo, tamanho_lote);
        if (dados_lote.empty()) {
            break;
        }

        std::vector<std::vector<std::string>> novos_dados(dados_lote.size(), std::vector<std::string>(cabecalho.size()));

        // Cria uma thread para fatorizar colunas
        std::thread fatoriza_thread(std::ref(id), std::ref(mapeamento), fatorizar_colunas_em_lotes, std::ref(dados_lote), std::ref(cabecalho), std::ref(novos_dados), std::ref(m));
        fatoriza_thread.join(); // Espera a thread terminar

        // Salvar unificado em um arquivo CSV
        salvar_csv_unificado(novos_dados, cabecalho, "merged_dataset.csv", escrever_cabecalho);
        escrever_cabecalho = false;
    }

    std::cout << "arquivo unificado salvo em 'merged_dataset.csv'\n";
    return 0;
}


Overwriting thread.cpp


In [ ]:
!g++ -o thread thread.cpp -std=c++11 -pthread
!./thread


A saída de streaming foi truncada nas últimas 5000 linhas.
Coluna 'sentido' fatorizada e salva em 'dataset_sentido.csv'
Coluna 'cdmercadoria_cntr' fatorizada e salva em 'dataset_cdmercadoria_cntr.csv'
.csv'
Coluna 'cdtup' fatorizada e salva em 'dataset_cdtup.csv'
Coluna 'berco' fatorizada e salva em 'dataset_berco.csv'
Coluna 'portoatracacao' fatorizada e salva em 'dataset_portoatracacao.csv'
Coluna 'mes' fatorizada e salva em 'dataset_mes.csv'
Coluna 'tipooperacao' fatorizada e salva em 'dataset_tipooperacao.csv'
Coluna 'tiponavegacaoatracacao' fatorizada e salva em 'dataset_tiponavegacaoatracacao.csv'
Coluna 'terminal' fatorizada e salva em 'dataset_terminal.csv'
Coluna 'origem' fatorizada e salva em 'dataset_origem.csv'
Coluna 'destino' fatorizada e salva em 'dataset_destino.csv'
Coluna 'cdmercadoria' fatorizada e salva em 'dataset_cdmercadoria.csv'
Coluna 'naturezacarga' fatorizada e salva em 'dataset_naturezacarga.csv'
Coluna 'sentido' fatorizada e salva em 'dataset_sentido.csv'
C

#VERSÃO OPEN MP

In [ ]:
%%writefile openmp_version.cpp
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <unordered_map>
#include <set>
#include <algorithm>
#include <omp.h>

// função para ler o CSV em lotes
std::vector<std::vector<std::string>> ler_csv_em_lotes(std::ifstream &arquivo, int tamanho_lote) {
    std::vector<std::vector<std::string>> dados;
    std::string linha, celula;
    int linhas_lidas = 0;

    while (linhas_lidas < tamanho_lote && std::getline(arquivo, linha)) {
        std::vector<std::string> linha_dados;
        std::stringstream linhaStream(linha);
        while (std::getline(linhaStream, celula, ',')) {
            linha_dados.push_back(celula);
        }
        dados.push_back(linha_dados);
        linhas_lidas++;
    }

    return dados;
}

// função para verificar se uma string é numérica
bool eh_numerico(const std::string &str) {
    char* endptr = 0;
    strtod(str.c_str(), &endptr);
    return *endptr == '\0' && !str.empty();
}

// função para fatorizar colunas nominais em lotes
std::vector<std::vector<std::string>> fatorizar_colunas_em_lotes(const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, std::vector<std::string> &novo_cabecalho) {
    std::vector<std::vector<std::string>> novos_dados(dados.size(), std::vector<std::string>(cabecalho.size()));

    #pragma omp parallel for
    for (size_t col = 0; col < cabecalho.size(); ++col) {
        // Verificar se a coluna é nominal
        bool eh_nominal = std::any_of(dados.begin(), dados.end(), [col](const std::vector<std::string> &linha) {
            return !linha[col].empty() && !eh_numerico(linha[col]);
        });

        if (eh_nominal) {
            std::unordered_map<std::string, int> mapeamento;
            int id = 0;

            for (size_t lin = 0; lin < dados.size(); ++lin) {
                if (!dados[lin][col].empty()) {
                    #pragma omp critical
                    {
                        if (mapeamento.find(dados[lin][col]) == mapeamento.end()) {
                            mapeamento[dados[lin][col]] = id++;
                        }
                    }
                    novos_dados[lin][col] = std::to_string(mapeamento[dados[lin][col]]);
                }
            }

            // sincronizar o acesso ao arquivo CSV individual
            std::string nome_arquivo = "dataset_" + cabecalho[col] + ".csv";
            #pragma omp critical
            {
                std::ofstream arquivo(nome_arquivo, std::ios::app);
                for (size_t lin = 0; lin < dados.size(); ++lin) {
                    arquivo << novos_dados[lin][col] << "\n";
                }
                arquivo.close();
            }
            std::cout << "Coluna '" << cabecalho[col] << "' fatorizada e salva em '" << nome_arquivo << "'\n";
        } else {
            for (size_t lin = 0; lin < dados.size(); ++lin) {
                novos_dados[lin][col] = dados[lin][col];
            }
        }
    }

    return novos_dados;
}

// Função para salvar unificado em um arquivo CSV
void salvar_csv_unificado(const std::vector<std::vector<std::string>> &dados, const std::vector<std::string> &cabecalho, const std::string &nome_arquivo, bool escrever_cabecalho) {
    std::ofstream arquivo(nome_arquivo, std::ios::app);

    // escrever cabeçalho
    if (escrever_cabecalho) {
        for (size_t i = 0; i < cabecalho.size(); ++i) {
            arquivo << cabecalho[i];
            if (i < cabecalho.size() - 1) {
                arquivo << ",";
            }
        }
        arquivo << "\n";
    }

    // escrever os dados
    for (const auto &linha : dados) {
        for (size_t i = 0; i < linha.size(); ++i) {
            arquivo << linha[i];
            if (i < linha.size() - 1) {
                arquivo << ",";
            }
        }
        arquivo << "\n";
    }

    arquivo.close();
}

int main() {
    std::string caminho = "/content/drive/MyDrive/Atividades_IA/dataset_00_sem_virg.csv";
    std::ifstream arquivo(caminho);
    std::vector<std::string> cabecalho;

    // tamanho do lote para leitura em lotes
    int tamanho_lote = 10000;
    bool escrever_cabecalho = true;

    // ler o cabeçalho
    std::string linha;
    if (std::getline(arquivo, linha)) {
        std::stringstream linhaStream(linha);
        std::string celula;
        while (std::getline(linhaStream, celula, ',')) {
            cabecalho.push_back(celula);
        }
    }

    // loop para processar o arquivo em lotes
    while (arquivo) {
        auto dados_lote = ler_csv_em_lotes(arquivo, tamanho_lote);
        if (dados_lote.empty()) {
            break;
        }

        std::vector<std::string> novo_cabecalho;
        auto novos_dados = fatorizar_colunas_em_lotes(dados_lote, cabecalho, novo_cabecalho);

        // salvar o arquivo unificado em um arquivo CSV
        salvar_csv_unificado(novos_dados, novo_cabecalho, "merged_dataset.csv", escrever_cabecalho);
        escrever_cabecalho = false;
    }

    std::cout << "arquivo unificado salvo em 'merged_dataset.csv'\n";
    return 0;
}



Writing openmp_version.cpp


In [ ]:
!g++ -fopenmp openmp_version.cpp -o openmp_version

!./openmp_version


A saída de streaming foi truncada nas últimas 5000 linhas.
Coluna 'cdmercadoria_cntr' fatorizada e salva em 'dataset_cdmercadoria_cntr.csv'
.csv'
Coluna 'cdtup' fatorizada e salva em 'dataset_cdtup.csv'
Coluna 'berco' fatorizada e salva em 'dataset_berco.csv'
Coluna 'portoatracacao' fatorizada e salva em 'dataset_portoatracacao.csv'
Coluna 'origem' fatorizada e salva em 'dataset_origem.csv'
Coluna 'mes' fatorizada e salva em 'dataset_mes.csv'
Coluna 'destino' fatorizada e salva em 'dataset_destino.csv'
Coluna 'tipooperacao' fatorizada e salva em 'dataset_tipooperacao.csv'
Coluna 'tiponavegacaoatracacao' fatorizada e salva em 'dataset_tiponavegacaoatracacao.csv'
Coluna 'cdmercadoria' fatorizada e salva em 'dataset_cdmercadoria.csv'
Coluna 'terminal' fatorizada e salva em 'dataset_terminal.csv'
Coluna 'naturezacarga' fatorizada e salva em 'dataset_naturezacarga.csv'
Coluna 'sentido' fatorizada e salva em 'dataset_sentido.csv'
Coluna 'cdmercadoria_cntr' fatorizada e salva em 'dataset_cdme